<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [4]:
from bs4 import BeautifulSoup
import requests
import time, os

import sys
import importlib

# webdriver imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [6]:
import functions as methods

In [8]:
# Configure window for multi-monitor automation
config_driver = webdriver.Chrome(chromedriver)

window_rect = (1450, -500, 900, 800) # x, y, width, height
config_driver.set_window_rect(*window_rect)

{'height': 800, 'width': 900, 'x': 1450, 'y': -360}

In [9]:
# End driver session after window is configured.
config_driver.quit()

In [46]:
def scroll_bottom(dr):
    '''
    
    Thwart infinite scroll. Takes driver as input
    Thanks to @ Matthew Eungoo Lee starting me off with this snippet.
    
    '''
    
    # Load Entire Page by Scrolling to the Bottom
    SCROLL_PAUSE_TIME = 2# Scroll to very Bottom to Load All
    last_height = dr.execute_script("return document.body.scrollHeight")# Get scroll height
    while True:    
        dr.execute_script(
            "window.scrollTo(0, document.body.scrollHeight);"
        ) # Scroll down to bottom

        time.sleep(0.5) # Wait a sec

        # Go back up, to trigger infinite scroll
        dr.execute_script(
            "window.scrollTo(0, 0);"
        ) # Scroll down to bottom

        time.sleep(0.5) # Wait a sec

        # Then back down
        dr.execute_script(
            "window.scrollTo(0, document.body.scrollHeight);"
        ) # Scroll down to bottom

        time.sleep(SCROLL_PAUSE_TIME) # Wait to load page

        new_height = dr.execute_script(
            "return document.body.scrollHeight"
        )# Calculate new scroll height and compare with last scroll height
        if new_height == last_height:
            return True
        last_height = new_height

In [57]:
# Open the category page fully, in preparation for scrape.

test_category_url = 'https://castbox.fm/categories/10028?country=us'

def get_category_source(url, dr):
    dr.set_window_rect(*window_rect)
    dr.get(url)
    try:
        # close annoying cookie verification
        dr.find_element_by_class_name('allow').click()
    except:
        # sometimes the cookie doesn't appear
        pass
    scroll_bottom(dr)
    return dr.page_source

driver = webdriver.Chrome(chromedriver)
print(get_category_source(test_category_url, driver))

<html lang="en" style="font-size: 100px;"><head>
      <link rel="canonical" href="https://castbox.fm/categories/10028?country=us">
      
      <title>Business</title>
      <meta charset="utf-8">
      <link rel="dns-prefetch" href="https//s3.castbox.fm">
      <link rel="dns-prefetch" href="https//everest.castbox.fm">
      <link rel="dns-prefetch" href="https//data.castbox.fm">
      <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
      <meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=no">
      
    
    <link rel="icon" href="https://s3.castbox.fm/app/castbox/static/images/logo_120.png" type="image/x-icon">
    <meta name="twitter:site" content="@CastBox_FM">
    <meta name="twitter:card" content="summary">
    <meta property="twitter:title" content="Business">
    <meta property="twitter:image" content="https://castbox.fm/app/castbox/static/images/logoshare.png">
    <meta property="twitter:description" content="Millions of podcast

In [62]:
def scrape_full_category_page(url, dr):
    '''
    
    Crawl through the entirety of a category page.
    Return the scraped category listing,
    as well as the name of the category.
    
    '''
    
    
    # open page properly and get full source
    source = get_category_source(url, dr)
    time.sleep(0.2)

    # grab entire listing as dictionary
    soup = BeautifulSoup(source, 'lxml')
    category_dict = methods.collect_category_pods(soup)
    
    # page category
    category_name = business_soup.find(class_='guru-breadcrumb-item active').text
    
    return category_name, category_dict

In [59]:
scrape_full_category_page(test_category_url, driver)

{'The Dave Ramsey Show': {'chan_url': 'https://castbox.fm//channel/The-Dave-Ramsey-Show-id2245',
  'author': 'Ramsey Network'},
 'How I Built This with Guy Raz': {'chan_url': 'https://castbox.fm//channel/How-I-Built-This-with-Guy-Raz-id431970',
  'author': 'NPR'},
 'RISE podcast': {'chan_url': 'https://castbox.fm//channel/RISE-podcast-id2010666',
  'author': 'Rachel Hollis'},
 'Planet Money': {'chan_url': 'https://castbox.fm//channel/Planet-Money-id431962',
  'author': 'NPR'},
 'Scam Goddess': {'chan_url': 'https://castbox.fm//channel/Scam-Goddess-id2361014',
  'author': 'Earwolf & Laci Mosley'},
 'The GaryVee Audio Experience': {'chan_url': 'https://castbox.fm//channel/The-GaryVee-Audio-Experience-id257325',
  'author': 'Gary Vaynerchuk'},
 'The Indicator from Planet Money': {'chan_url': 'https://castbox.fm//channel/The-Indicator-from-Planet-Money-id1101800',
  'author': 'NPR'},
 'Jocko Podcast': {'chan_url': 'https://castbox.fm//channel/Jocko-Podcast-id444695',
  'author': 'Jocko DEF

In [61]:
business_html = requests.get(test_category_url).text
business_soup = BeautifulSoup(business_html)
business_soup.find(class_='guru-breadcrumb-item active').text

'Business'

In [15]:
driver.quit()